In [1]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np
import math

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('helper_functions/')
# import util as util
import util

In [37]:
# Load the env file 
dotenv_path = './variables.env'

# Import the env variables
load_dotenv(dotenv_path, override=True)

schema = os.getenv("SCHEMA")
port = os.getenv("PORT")
host = os.getenv("HOST")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

# Create a connection to the database
db_eng = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}',
                       connect_args={'options': f'-csearch_path={schema}'},
                       isolation_level = 'SERIALIZABLE')


print("Successfully created db engine.")


Successfully created db engine.


In [2]:
from pymongo import MongoClient

client = MongoClient()

In [6]:
# I have (or will have) a database "airbnb"
db = client.airbnb


print('The list of all databases currently in the MongoDB client is:')
print(client.list_database_names())

print('\nThe list of all collections in the airbnb database is:')
print(db.list_collection_names())
# Note: calendar may not show up yet; it is created only when a first document is inserted into it

The list of all databases currently in the MongoDB client is:
['admin', 'airbnb', 'config', 'local', 'test']

The list of all collections in the airbnb database is:
['listings_with_reviews_and_cal', 'listings_with_calendar', 'listings', 'reviews', 'listings_with_reviews_m', 'calendar', 'listings_test']


In [3]:
def listing_convert_date_str_to_datetime(dt):
    if dt is None:
        return None
    elif pd.isnull(dt):  # tests whether dt is the pandas value NaT ("not a time")
        #print('\nEntered the NaT case\n')
        return None
    elif dt != dt:
        return None        # could also use math.nan, I think
    elif dt == '':
        return None
    else:
        month, day, year = dt.split('/')
        year = int(f'20{year}')
        day = int(day)
        month = int(month)
        temp = datetime(year, month, day)
        ts = temp.timestamp()
        new_dt = datetime.fromtimestamp(ts)
        return new_dt

In [4]:
# also converts NaT to None, because MongoDB does not recognize NaT
def convert_date_str_to_datetime(dt):
    if dt is None:
        return None
    elif pd.isnull(dt):  # tests whether dt is the pandas value NaT ("not a time")
        # print('\nEntered the NaT case\n')
        return None
    elif dt != dt:
        return None        # could also use math.nan, I think
    elif dt == '':
        return None
    else:
        year = int(dt[0:4])
        month = int(dt[5:7])
        day = int(dt[8:10])
        # print(year, month, day)
        temp = datetime(year, month, day)
        ts = temp.timestamp()
        new_dt = datetime.fromtimestamp(ts)
        return new_dt


### Loading listings.csv

In [42]:
listings_dtypes = {
    'id': str,
    'name': str,
    'host_id': str,
    'host_name': str,
    'neighbourhood_group': str,
    'neighbourhood': str,
    'room_type': str,
    'last_review': str,
    'license': str
}

listings_df = pd.read_csv('./listings.csv', dtype=listings_dtypes)

In [43]:
listings_df['last_review'] = listings_df['last_review'].apply(listing_convert_date_str_to_datetime)

In [44]:
listings_df['last_review'] = listings_df['last_review'].astype('object').where(listings_df['last_review'].notnull(), None)

### Loading reviews.csv

In [45]:
reviews_dtypes = {
    'listing_id': str,
    'id': str,
    'date': str,
    'reviewer_id': str,
    'reviewer_name': str,
    'comments': str
}

reviews_df = pd.read_csv('./reviews.csv', dtype=reviews_dtypes)

In [46]:
reviews_df['date'] = reviews_df['date'].apply(convert_date_str_to_datetime)

In [47]:
reviews_df['date'] = reviews_df['date'].astype('object').where(reviews_df['date'].notnull(), None)

### Put listings into MongoDB

In [48]:
time1 = datetime.now()
dict_listings = listings_df.to_dict('records')
time2 = datetime.now()
print(f'Time to perform this operation was {util.time_diff(time1,time2)} seconds.')

Time to perform this operation was 0.242995 seconds.


In [49]:
# The following empties out listings_with_cal; useful if making a fresh start
db.listings.drop()

print(len(dict_listings))

time1 = datetime.now()
result = db.listings.insert_many(dict_listings)
time2 = datetime.now()
print(f'\nTime to perform this operation was {util.time_diff(time1,time2)} seconds.')
# between about 2 and 4 minutes

print(f'\nNumber of docs in db.listings is {db.listings.count_documents({})}')

print()
outdocs = []
for o in result.inserted_ids[-5:]:
    outdocs.append(db.listings.find_one({ '_id': o}))
pprint.pp(outdocs)

39202

Time to perform this operation was 0.448996 seconds.

Number of docs in db.listings is 39202

[{'_id': ObjectId('665ea170e9a75767d052e090'),
  'id': '795691344180160853',
  'name': 'Hotel in New York · ★4.33 · 1 bedroom · 1 bed · 1 private bath',
  'host_id': '484277630',
  'host_name': 'Bugra Han',
  'neighbourhood_group': 'Manhattan',
  'neighbourhood': 'Midtown',
  'latitude': 40.7550775,
  'longitude': -73.9810571,
  'room_type': 'Private room',
  'price': nan,
  'minimum_nights': 4,
  'number_of_reviews': 5,
  'last_review': datetime.datetime(2023, 12, 18, 0, 0),
  'reviews_per_month': 1.2,
  'calculated_host_listings_count': 33,
  'availability_365': 0,
  'number_of_reviews_ltm': 5,
  'license': 'Exempt'},
 {'_id': ObjectId('665ea170e9a75767d052e091'),
  'id': '942191196511011206',
  'name': 'Home in Brooklyn · 5 bedrooms · 5 beds · 2 baths',
  'host_id': '517145594',
  'host_name': 'Ben',
  'neighbourhood_group': 'Brooklyn',
  'neighbourhood': 'East Flatbush',
  'latitude

In [50]:
db.listings.create_index('id')

'id_1'

### Put reviews into MongoDB

In [51]:
time1 = datetime.now()
dict_reviews = reviews_df.to_dict('records')
time2 = datetime.now()
print(f'Time to perform this operation was {util.time_diff(time1,time2)} seconds.')

Time to perform this operation was 4.842547 seconds.


In [52]:
# The following empties out reviews; useful if making a fresh start
db.reviews.drop()

print(len(dict_reviews))

time1 = datetime.now()
result = db.reviews.insert_many(dict_reviews)
time2 = datetime.now()
print(f'\nTime to perform this operation was {util.time_diff(time1,time2)} seconds.')
# between about 2 and 4 minutes

print(f'\nNumber of docs in db.reviews is {db.reviews.count_documents({})}')

print()
outdocs = []
for o in result.inserted_ids[-5:]:
    outdocs.append(db.reviews.find_one({ '_id': o}))
pprint.pp(outdocs)

986810

Time to perform this operation was 10.95913 seconds.

Number of docs in db.reviews is 986810

[{'_id': ObjectId('665ea178e9a75767d061ef4a'),
  'listing_id': '1066905373347759013',
  'id': '1076164094756359504',
  'date': datetime.datetime(2024, 1, 24, 0, 0),
  'reviewer_id': '84336263',
  'reviewer_name': 'T',
  'comments': 'We had a good stay. Place was clean and in a good, central '
              'location. It was a good value for the price and worked well for '
              'our group of 4 adults.'},
 {'_id': ObjectId('665ea178e9a75767d061ef4b'),
  'listing_id': '1066905373347759013',
  'id': '1079811744368032164',
  'date': datetime.datetime(2024, 1, 29, 0, 0),
  'reviewer_id': '280434951',
  'reviewer_name': 'Saufia',
  'comments': 'I recently had the pleasure of staying at this property and we '
              'had an amazing experience. The property has been recently '
              "refurbished and cleanliness was remarkable. It's perfect for a "
              "big grou

In [53]:
db.reviews.create_index('listing_id')

'listing_id_1'

### Building **listings_with_reviews_m**

In [54]:
# making sure that listings_with_calendar is empty
db.listings_with_reviews_m.drop()

pipeline = [
    {
        '$lookup':
        {
            'from': 'reviews',
            'localField': 'id',
            'foreignField': 'listing_id',
            'as': 'reviews'
        }
    },
    {
        '$project':
        {
            'reviews._id': 0,
        }
    },
    {
        '$out': 'listings_with_reviews_m'
    }
]

time1 = datetime.now()
test1 = db.listings.aggregate(pipeline)
time2 = datetime.now()
diff = util.time_diff(time1, time2)

print('\nTime it took was:', format(diff, '.4f'), '.')

print(db.list_collection_names())

#print("test1:")

#print(len(list(test1)))

#print(type(test1))


Time it took was: 6.3692 .
['listings', 'calendar', 'reviews', 'listings_with_reviews_m', 'listings_with_calendar', 'listings_test']


In [7]:
count = db.listings_with_reviews_m.count_documents({})
print(count)

39202


In [8]:
print(db.listings_with_reviews_m.count_documents({}))

cursor = db.listings_with_reviews_m.find({'id' : {'$regex' : '^1000.*$'}})
    
l = list(cursor)
print(len(l))

39202
43


In [9]:
def isnan(val):
    return val != val

In [10]:
def convert_lwrm_to_json(doc):
    doc_new = {}
    
    doc_new['_id'] = str(doc['_id'])
    
    for key in ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews']:
        if not isnan(doc[key]):
            doc_new[key] = doc[key]
        else:
            doc_new[key] = None

    if doc['last_review']:
        doc_new['last_review'] = doc['last_review'].strftime('%Y-%m-%d')
    else:
        doc_new['last_review'] = None
    
    for key in ['reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm']:
        if not isnan(doc[key]):
            doc_new[key] = doc[key]
        else:
            doc_new[key] = None

    if not isnan(doc['license']):
        doc_new['license'] = doc['license']
    else:
        doc_new['license'] = ""

    dlist = []
    for d in doc['reviews']:
        d_new = {}
        for key in ['listing_id', 'id']:
            if not isnan(d[key]):
                d_new[key] = d[key]
            else:
                d_new[key] = None

        if d['date']:
            d_new['date'] = d['date'].strftime('%Y-%m-%d')
        else:
            d_new['date'] = None

        for key in ['reviewer_id', 'reviewer_name', 'comments']:
            if not isnan(d[key]):
                d_new[key] = d[key]
            else:
                d_new[key] = None

        dlist.append(d_new)

    doc_new['reviews'] = dlist

    return doc_new

In [11]:
cursor = db.listings_with_reviews_m.find({'id' : {'$regex' : '^1000.*$'}})

output = []

for doc in cursor:
    output.append(convert_lwrm_to_json(doc))

print(len(output))

43


In [12]:
# Writing dict to a json file into a json file in a subdirectory
# Also putting this function into my util.py
def write_dict_to_dir_json(dict, dir, filename):
    with open(dir + '/' + filename, 'w') as fp:
        json.dump(dict, fp)

dir = 'OUTPUTS'
filename = 'listings_with_reviews_m_subset_1000.json'
write_dict_to_dir_json(output, dir, filename)